In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
!pip install category_encoders
from fastai.tabular import *
from IPython import display
display.clear_output()

In [0]:
df = pd.read_csv('doctor_train.csv')
tdf = pd.read_csv('doctor_test.csv')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [6]:
# df.info()
def dataessing(df, test):
  cat_col = ['Profession','Status','edu','Irregular','residence','prev_diagnosed','communication','Month','side_effects']
  num_col = ['age','day','Time','Doctor_visits','last_visit','cured_in']
  loan = ['Money']
  target = 'Y'
  train_DF = pd.DataFrame(); valid_DF = pd.DataFrame(); test_DF = pd.DataFrame();
  dfs = [df, test]
  for i in range(0,2):
    d = dfs[i]
    dfc = pd.DataFrame()
    dfc[cat_col] = d[cat_col].fillna("unknown")
    dfc[num_col+loan] = d[num_col+loan].fillna(0)
    dfc[num_col] = dfc[num_col].clip(lower=0)
    dfcd = dfc
    if i == 0:
      from sklearn.model_selection import train_test_split
      train_DF, valid_DF, y_train, y_test = train_test_split(dfcd, df[target], random_state=45, test_size=0.3)
      train_DF['Y'] = y_train
      valid_DF['Y'] = y_test
    else:
      test_DF = dfcd
  return train_DF, valid_DF, test_DF, num_col, cat_col, target

train_DF, valid_DF, test_DF, num_col, cat_col, target = dataessing(df, tdf)

import category_encoders as ce
cb_enc = ce.CatBoostEncoder(cols=cat_col, random_state=7)
cb_enc.fit(train_DF[cat_col], train_DF['Y'])
train_DF[cat_col] = cb_enc.transform(train_DF[cat_col])
valid_DF[cat_col] = cb_enc.transform(valid_DF[cat_col])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [95]:
train_data = train_DF
test_data = valid_DF
num_col = num_col
cat_col = cat_col
dep_var = target
procs = [FillMissing, Categorify, Normalize]

data = (TabularList.from_df(train_data, cont_names=num_col, cat_names=cat_col, procs=procs)
          .split_by_idx(valid_idx=range(int(len(train_data)*0.9),len(train_data)))
          .label_from_df(cols=dep_var)
          .add_test(TabularList.from_df(test_data, cont_names=num_col, cat_names=cat_col, procs=procs))
          .databunch())

# learn = tabular_learner(data, layers=[100, 50, 50, 10], ps=[0.001,0.01,0.01,0.1], emb_drop=0.02, metrics=accuracy)
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.573414,0.540893,0.829787,00:12
1,0.513323,0.499390,0.858156,00:12
2,0.466769,0.448866,0.872847,00:12
3,0.424680,0.409128,0.855116,00:12
4,0.384882,0.398564,0.882472,00:12
5,0.358848,0.333312,0.895643,00:12
6,0.328063,0.314600,0.878926,00:12
7,0.307290,0.298093,0.883992,00:12
8,0.280974,0.282540,0.895643,00:13
9,0.285781,0.270878,0.902229,00:12


Buffered data was truncated after reaching the output size limit.

In [92]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)
y_hat = pred_class
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(accuracy_score(valid_DF['Y'], y_hat))
print(confusion_matrix(valid_DF['Y'], y_hat))
print(classification_report(valid_DF['Y'], y_hat))

0.9054261733065374
[[7265  226]
 [ 574  394]]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7491
           1       0.64      0.41      0.50       968

    accuracy                           0.91      8459
   macro avg       0.78      0.69      0.72      8459
weighted avg       0.89      0.91      0.90      8459

